<a href="https://colab.research.google.com/github/takumi19910112/google_colab/blob/main/fine_tuning_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Google Colab の場合は上記の環境構築手順を行なわず、単にこのセルから実行していってください。
!pip uninstall unsloth -y
!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-ggmv90u5/unsloth_55058d63f5ea42818435723dbdc56a26
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-ggmv90u5/unsloth_55058d63f5ea42818435723dbdc56a26
  Resolved https://github.com/unslothai/unsloth.git to commit 8558bc92b06f9128499484ef737fa71b966ffc23
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 188.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 333.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 283.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 kB 250.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 314.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 333.2 MB/s eta 0:00

In [ ]:
# Google Colab のデフォルトで入っているパッケージをアップグレード
!pip install --upgrade torch
!pip install --upgrade xformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 53.5 MB/s eta 0:00:00


In [ ]:
# notebookでインタラクティブな表示を可能とする（ただし、うまく動かない場合あり）
!pip install ipywidgets --upgrade

# Install Flash Attention 2 for softcapping support
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 73.1 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.10
    Uninstalling widgetsnbextension-3.6.10:
      Successfully uninstalled widgetsnbextension-3.6.10
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully uninstalled ipywidgets-7.7.1


In [ ]:
# モデルを作成する場合はこっち
# # llm-jp/llm-jp-3-13bを4bit量子化のqLoRA設定でロード。

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from unsloth import FastLanguageModel
import torch
from google.colab import drive
drive.mount('/content/drive')
max_seq_length = 512 # 入力シーケンスの最大長さを設定します
dtype = None # データタイプ（Noneの場合、自動的に設定されます）
load_in_4bit = True # 4bit量子化してメモリの節約

model_id = "takumi0112/llm-jp-3-13b-finetune-2"
new_model_id = "llm-jp-3-13b-finetune-it" #Fine-Tuningしたモデルにつけたい名前
"""
LoRA（Low-Rank Adaptation）は、大規模モデルを効率的にファインチューニングする手法です。
target_modulesには、LoRAを適用するモデル内のモジュールを指定します。
use_gradient_checkpointingを使うと、メモリ使用量を削減できます。
"""
# FastLanguageModel インスタンスを作成
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

# SFT用のモデルを用意
model = FastLanguageModel.get_peft_model(
    model,
    r = 32,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0.05,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
    max_seq_length = max_seq_length,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Mounted at /content/drive
Unsloth: WARNING `trust_remote_code` is True.
Are you certain you want to do remote code execution?
==((====))==  Unsloth 2024.11.10: Fast Llama patching. Transformers:4.46.2.
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/494 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Unsloth 2024.11.10 patched 40 layers with 0 QKV layers, 0 O layers and 0 MLP layers.
Unsloth: Already have LoRA adapters! We shall skip this step.


In [ ]:
# もしモデルを保存する場合はこれを実行
# 保存先ディレクトリをGoogle ドライブ内に指定
import os
save_dir = '/content/drive/MyDrive/MyModel/'

# ディレクトリが存在しない場合は新しく作成
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

# モデルの保存
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)


('/content/drive/MyDrive/MyModel/tokenizer_config.json',
 '/content/drive/MyDrive/MyModel/special_tokens_map.json',
 '/content/drive/MyDrive/MyModel/tokenizer.json')

In [ ]:
# # 保存したモデルの場合はこっち
# from google.colab import drive
# drive.mount('/content/drive')
# from transformers import AutoModelForCausalLM, AutoTokenizer

# # 保存したディレクトリのパス
# save_dir = '/content/drive/MyDrive/MyModel/'

# # モデルとトークナイザのロード
# model = AutoModelForCausalLM.from_pretrained(save_dir)
# tokenizer = AutoTokenizer.from_pretrained(save_dir)

Mounted at /content/drive


config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/29.9k [00:00<?, ?B/s]

model-00001-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00003-of-00006.safetensors:   0%|          | 0.00/4.88G [00:00<?, ?B/s]

model-00004-of-00006.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
from google.colab import userdata
!ls /content/drive/MyDrive/huggingface/hub/
# from google.colab import userdata
HF_TOKEN=userdata.get('HF_TOKEN')

datasets--elyza--ELYZA-tasks-100  models--google--gemma-2-2b-jpn-it  version.txt
huggingface_models.tar.gz	  original


In [ ]:
# 学習に用いるデータセットの指定
# 今回はLLM-jp の公開している Ichikara Instruction を使います。データにアクセスするためには申請が必要ですので、使いたい方のみ申請をしてください。
# Ichikara Instruciton を Hugging Face Hub にて公開することはお控えください。

# 下記のリンクから申請を終えた先に Google Drive があり、Distribution20241221_all というフォルダごとダウンロードしてください。
# 今回は「ichikara-instruction-003-001-1.json」を使います。必要であれば展開（!unzip など）し、データセットのパスを適切に指定してください。
# omnicampusの開発環境では取得したデータを左側にドラッグアンドドロップしてお使いください。
# Google Colab の場合も左のサイドバーよりドラッグ&ドロップでアップデートしてください。

# https://liat-aip.sakura.ne.jp/wp/llmのための日本語インストラクションデータ作成/llmのための日本語インストラクションデータ-公開/
# 関根聡, 安藤まや, 後藤美知子, 鈴木久美, 河原大輔, 井之上直也, 乾健太郎. ichikara-instruction: LLMのための日本語インストラクションデータの構築. 言語処理学会第30回年次大会(2024)

from datasets import load_dataset

dataset = load_dataset("json", data_files="/content/drive/MyDrive/Distribution20241221_all/ichikara-instruction-003-001-1.json")
# dataset = load_dataset("json", data_files="/content/drive/MyDrive/elyza_tasks_modified.jsonl")
# 今回はELYZA-tasks-100で学習
# dataset = load_dataset("elyza/ELYZA-tasks-100")
# dataset = load_dataset("DeL-TaiseiOzaki/news_summary_2024secondhalf")
# パスの指定にご注意ください。アップロードしたファイルを右クリックし、「パスをコピー」をクリック、上記の data_files と合致していることをご確認ください。Omnicampus のディレクトリ構造とは異なるかもしれません。

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
# from datasets import load_dataset

# # データセットの読み込み
# dataset = load_dataset("elyza/ELYZA-tasks-100")
# print(dataset)

# 学習時のプロンプトフォーマットの定義
prompt = """### 指示
{}
### 回答
{}"""

# トークナイザーのEOSトークン（文末トークン）
EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    input_text = examples["input"]  # "input" フィールドが入力データ
    output_text = examples["output"]  # "output" フィールドが出力データ
    text = prompt.format(input_text, output_text) + EOS_TOKEN  # プロンプトの作成
    return { "formatted_text" : text }  # 新しいフィールド "formatted_text" を返す

# データセット全体にフォーマット関数を適用
dataset = dataset.map(
    formatting_prompts_func,
    num_proc=4,  # 並列処理数を指定
)
print(dataset)

Map (num_proc=4):   0%|          | 0/100 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['task_id', 'input', 'output', 'formatted_text'],
        num_rows: 100
    })
})


In [ ]:

# from datasets import load_dataset

# dataset = load_dataset("DeL-TaiseiOzaki/news_summary_2024secondhalf")
# # プロンプトフォーマットの定義
# # 特に "summary" に対するプロンプトフォーマットを作成
# prompt = """### 要約
# {}
# ### 終了"""

# EOS_TOKEN = tokenizer.eos_token

# def formatting_prompts_func(examples):
#     summary_text = examples["summary"]  # "summary" フィールドが要約データ
#     text = prompt.format(summary_text) + EOS_TOKEN  # プロンプトの作成
#     return { "formatted_text" : text }  # 新しいフィールド "formatted_text" を返す


# # データセット全体にフォーマット関数を適用
# dataset = dataset.map(
#     formatting_prompts_func,
#     num_proc=4,  # 並列処理数を指定
# )

# # フォーマットが適用されたデータセットを確認
# print(dataset)

Map (num_proc=4):   0%|          | 0/39998 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['summary', 'formatted_text'],
        num_rows: 39998
    })
})


In [ ]:
# """
# プロンプトのフォーマットを統一することで、モデルが学習しやすくなります。
# 前処理関数で、各データを指定のフォーマットに変換し、新しいフィールドformatted_textに保存します。
# """
# # 学習時のプロンプトフォーマットの定義
# prompt = """### 指示
# {}
# ### 回答
# {}"""



# """
# formatting_prompts_func: 各データをプロンプトに合わせた形式に合わせる
# """
# EOS_TOKEN = tokenizer.eos_token # トークナイザーのEOSトークン（文末トークン）
# def formatting_prompts_func(examples):
#     input = examples["text"] # 入力データ
#     output = examples["output"] # 出力データ
#     text = prompt.format(input, output) + EOS_TOKEN # プロンプトの作成
#     return { "formatted_text" : text, } # 新しいフィールド "formatted_text" を返す
# pass

# # # 各データにフォーマットを適用
# dataset = dataset.map(
#     formatting_prompts_func,
#     num_proc= 4, # 並列処理数を指定
# )

# dataset

Map (num_proc=4):   0%|          | 0/1729 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'output', 'formatted_text'],
        num_rows: 1729
    })
})

In [ ]:
# データを確認
print(dataset["train"]["formatted_text"][3])

### 指示
以下の説明を読んだユーザーが感じるであろう疑問とは？

大量のゴミの処理にお困りではありませんか？

「生産で出るゴミ処理費用が大幅にかかっている」
「指定ゴミ袋が導入され、一気にコストが上がってしまった」
「工場が狭いので、ゴミ保管スペースを減らしたい」
「ISO14001でゴミ削減が大きな課題となっている」

場所をとる嵩張るゴミをつぶしてまとめて運びやすい形にすることで、処理・保管・運搬にかかる様々なコストを削減したり、環境負荷の低減にも貢献できるのが、廃棄物圧縮減容機→『ゴミ圧縮機』です。

ゴミ圧縮機の主な用途は廃プラスチック・紙・段ボール・袋類・ペットボトル・プラスチック容器などつぶれるが元に戻ってしまうゴミを圧縮梱包します。
### 回答
* どのくらいのサイズ・種類のゴミ圧縮機があるの？導入費用はどれくらいかかるの？
* 圧縮後のゴミの処理方法は？最終処分場への搬入はどうなるの？
* 圧縮機のメンテナンスはどうするの？故障した場合の対応は？
* 私たちの工場のゴミ量や種類に合った機種を選べるの？導入前にテストできるの？
* 圧縮率はどのくらい？どれだけの費用削減効果が期待できるの？具体的な事例は？
* ISO14001認証取得に向けたサポートはしてくれるの？
* 電力消費量はどのくらい？ランニングコストは？
* ゴミ圧縮機を設置するのに必要なスペースはどのくらい？
* 安全面はどうなっているの？作業中の事故リスクは？
* リースやレンタルのプランはあるの？</s>


In [ ]:
"""
training_arguments: 学習の設定

  - output_dir:
      -トレーニング後のモデルを保存するディレクトリ

  - per_device_train_batch_size:
      - デバイスごとのトレーニングバッチサイズ

  - per_device_eval_batch_size:
      - デバイスごとの評価バッチサイズ

  - gradient_accumulation_steps:
      - 勾配を更新する前にステップを積み重ねる回数

  - optim:
      - オプティマイザの設定

  - num_train_epochs:
      - エポック数

  - eval_strategy:
      - 評価の戦略 ("no"/"steps"/"epoch")

  - eval_steps:
      - eval_strategyが"steps"のとき、評価を行うstep間隔

  - logging_strategy:
      - ログ記録の戦略

  - logging_steps:
      - ログを出力するステップ間隔

  - warmup_steps:
      - 学習率のウォームアップステップ数

  - save_steps:
      - モデルを保存するステップ間隔

  - save_total_limit:
      - 保存しておくcheckpointの数

  - max_steps:
      - トレーニングの最大ステップ数

  - learning_rate:
      - 学習率

  - fp16:
      - 16bit浮動小数点の使用設定（第8回演習を参考にすると良いです）

  - bf16:
      - BFloat16の使用設定

  - group_by_length:
      -  入力シーケンスの長さによりバッチをグループ化 (トレーニングの効率化)

  - report_to:
      - ログの送信先 ("wandb"/"tensorboard"など)
"""
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
"""
TrainingArgumentsでトレーニングの詳細な設定を行います。
per_device_train_batch_sizeやgradient_accumulation_stepsでバッチサイズを調整します。
16ビット精度（fp16やbf16）を使うことで、メモリ使用量を削減し、計算を高速化します。

"""
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset=dataset["train"],
    max_seq_length = max_seq_length,
    dataset_text_field="formatted_text",
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        num_train_epochs = 1,
        logging_steps = 10,
        warmup_steps = 10,
        save_steps=100,
        save_total_limit=2,
        max_steps=-1,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        group_by_length=True,
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#@title 現在のメモリ使用量を表示
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.748 GB.
9.209 GB of memory reserved.


In [ ]:
#@title 学習実行
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 100 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 12
 "-____-"     Number of trainable parameters = 125,173,760


Step,Training Loss
10,1.844000


In [ ]:
# ELYZA-tasks-100-TVの読み込み。事前にファイルをアップロードしてください
# データセットの読み込み。
# omnicampusの開発環境では、左にタスクのjsonlをドラッグアンドドロップしてから実行。
import json
datasets = []
with open("/content/drive/MyDrive/elyza-tasks-100-TV_0.jsonl", "r") as f:
    item = ""
    for line in f:
      line = line.strip()
      item += line
      if item.endswith("}"):
        datasets.append(json.loads(item))
        item = ""

In [ ]:
# 学習したモデルを用いてタスクを実行
from tqdm import tqdm

# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
for dt in tqdm(datasets):
  input = dt["input"]

  prompt = f"""### 指示\n{input}\n### 回答\n"""

  inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, do_sample=False, repetition_penalty=1.2)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]

  results.append({"task_id": dt["task_id"], "input": input, "output": prediction})

100%|██████████| 100/100 [07:42<00:00,  4.62s/it]


In [ ]:
# jsonlで保存
with open(f"{new_model_id}_output_2024_11_30.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')

In [ ]:
!ls /content/
!pip install shutil  # This won't work, shutil is part of the standard library. See below.
import shutil
shutil.copy('/content/llm-jp-3-13b-finetune-it-2_output_2024_11_30.jsonl', '/content/drive/My Drive/')

drive			      llm-jp-3-13b-finetune-it-2_output_2024_11_30.jsonl  sample_data
huggingface_tokenizers_cache  outputs
ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


'/content/drive/My Drive/llm-jp-3-13b-finetune-it-2_output_2024_11_30.jsonl'

In [ ]:
# モデルとトークナイザーをHugging Faceにアップロード。
# 一旦privateでアップロードしてください。
# 最終成果物が決まったらpublicにするようお願いします。
# 現在公開しているModel_Inference_Template.ipynbはunslothを想定していないためそのままでは動かない可能性があります。
model.push_to_hub_merged(
    new_model_id,
    tokenizer=tokenizer,
    save_method="lora",
    token=HF_TOKEN,
    private=True
)

model.push_to_hub(new_model_id, token=HF_TOKEN, private=True) # Online saving
tokenizer.push_to_hub(new_model_id, token=HF_TOKEN) # Online saving

Unsloth: Saving LoRA adapters. Please wait...


README.md:   0%|          | 0.00/561 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

Saved lora model to https://huggingface.co/llm-jp-3-13b-finetune-it-2


No files have been modified since last commit. Skipping to prevent empty commit.


Saved model to https://huggingface.co/llm-jp-3-13b-finetune-it-2


No files have been modified since last commit. Skipping to prevent empty commit.
